In [1]:
#Data preprocessing important libraries
import pandas as pd  #Data manipulation
import argparse
import numpy as np
import pickle  
import nltk  #Natural language processing 
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords  #Stop word removal
from nltk.tokenize import word_tokenize  #Tokenizition
from nltk.tokenize import RegexpTokenizer
import pyarabic.araby as araby
import qalsadi.lemmatizer
import matplotlib.pyplot as plt
from collections import Counter
from nltk.util import ngrams
import matplotlib.pyplot as plt
import re
import string  #Remove punctuation & characters

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\s7r_2\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\s7r_2\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
text = pd.read_excel(r'C:\Users\s7r_2\Downloads\��تصنيف الاخبار العربيه - نسخة.xlsx')
text.head()

,رقم التصنيف,صنف الخبر,عنوان الخبر,clean,cleaned_stopwords,lemmatized_text,lemmatized
0,0,صحة,الجلوس أكثر من 10 ساعات يومياً يفاقم مخاطر الوفاة,الجلوس اكثر من ساعات يومياً يفاقم مخاطر الوفاه,الجلوس اكثر ساعات يوميا يفاقم مخاطر الوفاه,"[('جلوس', 'noun'), ('كثر', 'verb'), ('ساعة', '...",جلوس كثر ساعة يوم فاقم مخاطر الوفاه
1,5,رياضة,مولر: هناك طريقة واحدة لإيقاف خطورة ميسي,مولر هناك طريقه واحده لايقاف خطوره ميسي,مولر طريقه واحده لايقاف خطوره ميسي,"[('مولر', 'all'), ('طريق', 'noun'), ('واحد', '...",مولر طريق واحد لايقاف خطوره ميس
2,6,اقتصاد,"وزير مالية لبنان: ""إجراءات تقشفية استثنائية"" ب...",وزير ماليه لبنان اجراءات تقشفيه استثنائيه بموا...,وزير ماليه لبنان اجراءات تقشفيه استثنائيه بموازنه,"[('زير', 'noun'), ('مال', 'noun'), ('لبن', 'no...",زير مال لبن اجراءات تقشف استثناء موازن
3,4,فن,مروان خوري: هكذا أقضي فترة الحجر المنزلي,مروان خوري هكذا اقضي فتره الحجر المنزلي,مروان خوري اقضي فتره الحجر المنزلي,"[('مرو', 'noun'), ('خور', 'noun'), ('قضى', 've...",مرو خور قضى فتر حجر منزل
4,6,اقتصاد,مصر تبدأ حملة التطعيم بلقاح كورونا لعامة الشعب,مصر تبدا حمله التطعيم بلقاح كورونا لعامه الشعب,مصر تبدا حمله التطعيم بلقاح كورونا لعامه الشعب,"[('مصر', 'noun'), ('بد', 'verb'), ('حمل', 'nou...",مصر بد حمل تطعيم لقاح كار عام شعب


In [6]:
#Helper Functions
def most_common_words(tokenized_text, top_n=200):
    words = Counter(tokenized_text)
    return words.most_common(top_n)

def least_common_words(tokenized_text, bottom_n=200):
    words = Counter(tokenized_text)
    return words.most_common()[:-bottom_n-1:-1]

def most_common_ngrams(tokenized_text, n=2, top_n=200):
    ngrams_list = ngrams(tokenized_text, n)
    ngrams_count = Counter(ngrams_list)
    return ngrams_count.most_common(top_n)

word2count = {}

for data in text['lemmatized']:  
    words = nltk.word_tokenize(data) # Split text into words (tokens)
    for word in words:
        if word not in word2count:
            word2count[word] = 1
        else:
            word2count[word] += 1

# Convert dictionary to DataFrame
word_count_df = pd.DataFrame(word2count.items(), columns=['Word', 'Count'])

# Use auxiliary functions
tokenized_text = nltk.word_tokenize(' '.join(text['lemmatized']))  # جمع جميع النصوص
print("Most common words:")
print(most_common_words(tokenized_text))

print("Least common words:")
print(least_common_words(tokenized_text))

print("Most common 2-grams:")
print(most_common_ngrams(tokenized_text, n=2))

Most common words:
[('علي', 6489), ('ال', 3774), ('؟', 3062), ('مصر', 2942), ('كار', 2832), ('جديد', 2719), ('السعوديه', 2183), ('مليار', 1774), ('عالم', 1580), ('ب', 1295), ('سبب', 1269), ('اقتصاد', 1237), ('عام', 1205), ('مليون', 1096), ('دب', 1092), ('دور', 1074), ('طير', 1043), ('سب', 1003), ('ايران', 924), ('الامارات', 924), ('اميركا', 922), ('شهر', 879), ('بريطاني', 872), ('تطبيق', 870), ('امام', 861), ('صور', 839), ('ترمب', 823), ('خلال', 822), ('اتحاد', 818), ('سعودي', 790), ('ترك', 778), ('لصى', 776), ('بلا', 772), ('اتفاق', 765), ('لقاح', 753), ('فن', 745), ('اول', 742), ('طلق', 740), ('توقع', 738), ('يوم', 732), ('عراق', 727), ('رئيس', 717), ('حذر', 703), ('عمل', 697), ('علن', 688), ('دعم', 676), ('عرب', 660), ('سياح', 651), ('استثمار', 629), ('تعرف', 619), ('ل', 617), ('دول', 613), ('رحلة', 612), ('هاتف', 607), ('فيروس', 598), ('قطاع', 595), ('نمو', 593), ('مطار', 592), ('كثر', 579), ('لاعب', 579), ('زير', 575), ('عبر', 573), ('تراجع', 570), ('نظام', 567), ('حول', 564), ('ا

In [3]:
from sklearn.feature_extraction.text import CountVectorizer 
text = pd.read_excel(r'C:\Users\s7r_2\Downloads\��تصنيف الاخبار العربيه - نسخة.xlsx')
#Bag of Words (BoW)
bow_vectorizer = CountVectorizer(min_df=5)
bow_matrix = bow_vectorizer.fit_transform(text['lemmatized'])

#Convert to DataFrame for better visualization
bow_df = pd.DataFrame(bow_matrix.toarray(), columns=bow_vectorizer.get_feature_names_out())
print("Bag of Words (BoW) Matrix:")
print(bow_df)

Bag of Words (BoW) Matrix:
       aa  apple  arab  arabs  bbm  bmw  bollywood  brexit  cia  dna  ...  \
0       0      0     0      0    0    0          0       0    0    0  ...   
1       0      0     0      0    0    0          0       0    0    0  ...   
2       0      0     0      0    0    0          0       0    0    0  ...   
3       0      0     0      0    0    0          0       0    0    0  ...   
4       0      0     0      0    0    0          0       0    0    0  ...   
...    ..    ...   ...    ...  ...  ...        ...     ...  ...  ...  ...   
57782   0      0     0      0    0    0          0       0    0    0  ...   
57783   0      0     0      0    0    0          0       0    0    0  ...   
57784   0      0     0      0    0    0          0       0    0    0  ...   
57785   0      0     0      0    0    0          0       0    0    0  ...   
57786   0      0     0      0    0    0          0       0    0    0  ...   

       يوم  يونان  يونايتد  يونس  يونغ  يونيسف  

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Create the TfidfVectorizer
tfidf_Arabic = TfidfVectorizer()

# Apply fit_transform to the data
tfidf_matrix = tfidf_Arabic.fit_transform(text['lemmatized'])

print("TF-IDF Matrix shape:", tfidf_matrix.shape)
with open("tfidf_Arabic.pkl", "wb") as tfidf_file:
    pickle.dump(tfidf_Arabic,tfidf_file)

TF-IDF Matrix shape: (57787, 23529)


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

X = tfidf_matrix
y = text['صنف الخبر'] 

#Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, y, test_size=0.2, random_state=42)  #Split data

#Function to evaluate model performance
def evaluate_model(model, X_test, y_test):
    predictions = model.predict(X_test)  #Make predictions
    accuracy = accuracy_score(y_test, predictions)  #Calculate accuracy
    report = classification_report(y_test, predictions)  #Generate classification report
    return accuracy, report

svm_model = SVC(kernel='linear', class_weight='balanced')  #Initialize SVM model with linear kernel and class weights
svm_model.fit(X_train, y_train)  #Train the model
svm_accuracy, svm_report = evaluate_model(svm_model, X_test, y_test)  #Evaluate SVM
print("SVM Accuracy:", svm_accuracy)  #Print accuracy
print("SVM Classification Report:\n", svm_report)  #Print classification report

nb_model = MultinomialNB()  #Initialize Naive Bayes model
nb_model.fit(X_train, y_train)  #Train the model
nb_accuracy, nb_report = evaluate_model(nb_model, X_test, y_test)  #Evaluate Naive Bayes
print("Naive Bayes Accuracy:", nb_accuracy)  #Print accuracy
print("Naive Bayes Classification Report:\n", nb_report)  #Print classification report

rf_model = RandomForestClassifier(class_weight='balanced', random_state=42)  #Initialize Random Forest model with class weights
rf_model.fit(X_train, y_train)  #Train the model
rf_accuracy, rf_report = evaluate_model(rf_model, X_test, y_test)  #Evaluate Random Forest
print("Random Forest Accuracy:", rf_accuracy)  #Print accuracy
print("Random Forest Classification Report:\n", rf_report)  #Print classification report
with open("model_Arabic.pickle", "rb") as model_file:
    lg_model = pickle.load(model_file)

SVM Accuracy: 0.9103651150718117
SVM Classification Report:
               precision    recall  f1-score   support

      اقتصاد       0.89      0.89      0.89      2113
 التكنولوجيا       0.92      0.91      0.92      1499
     السياحة       0.90      0.91      0.91      1029
       رياضة       0.98      0.96      0.97      1956
       سياسة       0.85      0.85      0.85      1481
         صحة       0.89      0.93      0.91      1511
          فن       0.92      0.91      0.91      1969

    accuracy                           0.91     11558
   macro avg       0.91      0.91      0.91     11558
weighted avg       0.91      0.91      0.91     11558

Naive Bayes Accuracy: 0.8857068697006403
Naive Bayes Classification Report:
               precision    recall  f1-score   support

      اقتصاد       0.75      0.95      0.84      2113
 التكنولوجيا       0.94      0.87      0.90      1499
     السياحة       0.95      0.71      0.81      1029
       رياضة       0.96      0.96      0.96     

In [9]:
from sklearn.linear_model import LogisticRegression

# Data processing function
def preprocess_data(text):
    text = text.lower()  # تحويل النص إلى حروف صغيرة
    text = re.sub(r'[^\w\s\'\"]', '', text)  # إزالة علامات الترقيم
    return text

# Load data
text = text[text['lemmatized'].notnull() & (text['lemmatized'] != '')]

# Data processing
text['lemmatized'] = text['lemmatized'].apply(preprocess_data)

# Create a TF-IDF matrix
tfidf_vectorizer = TfidfVectorizer(min_df=5)
tfidf_matrix = tfidf_vectorizer.fit_transform(text['lemmatized'])

X = tfidf_matrix
y = text['صنف الخبر']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train logistic regression model
lg_model = LogisticRegression(max_iter=200, verbose=1)
lg_model.fit(X_train, y_train)

# Store the form and TF-IDF
with open("tfidf_Arabic.pkl", "rb") as tfidf_file:
    tfidf_Arabic = pickle.load(tfidf_file)
with open("model_Arabic.pickle", "rb") as model_file:
    svm_model_ar = pickle.load(model_file)

# Prediction function
def predict_category(text):
    cleaned_text = preprocess_data(text)  # معالجة النص
    tfidf_data = tfidf_vectorizer.transform([cleaned_text])  # تحويل النص إلى تمثيل TF-IDF
    prediction = lg_model.predict(tfidf_data)  # إجراء التنبؤ
    return prediction[0]  # إرجاع الفئة المتوقعة

# New text class prediction
new_text = "وزير مالية لبنان:إجراءات تقشفية استثنائية بموازنة 2019"
predicted_category = predict_category(new_text)
print(f"The predicted category is: {predicted_category}")

The predicted category is: اقتصاد


In [5]:
import os
import sys
import pickle
from pathlib import Path
import tkinter as tk
from tkinter import messagebox

# Load the model and TF-IDF Vectorizer
#Arabic
with open("tfidf_Arabic.pkl", "rb") as tfidf_file:
    tfidf_Arabic = pickle.load(tfidf_file)
with open("model_Arabic.pickle", "rb") as model_file:
    svm_model_ar = pickle.load(model_file)

# Text preprocessing function
def preprocess_data(text):
    text = text.lower()
    text = re.sub(r'[^\w\s\'\"]', '', text)
    return text

# Classification function using the loaded model
def predict_category(text):
    cleaned_text = preprocess_data(text)  # Text processing
    tfidf_data = tfidf_Arabic.transform([cleaned_text])  # Convert text to TF-IDF representation
    prediction = svm_model_ar.predict(tfidf_data)  # Make a prediction
    return prediction[0]  # Returns the expected class


# Function to classify text and show the result
def classify_text():
    text = entry.get()
    if not text.strip():
        messagebox.showwarning("خطأ", "الرجاء إدخال نص للتصنيف!")
        return
    category = predict_category(text)
    result_label.config(text=f"نوع الخبر : {category}")

# Function to clear text
def clear_text():
    entry.delete(0, tk.END)
    result_label.config(text="")

# Set up the main GUI window
root = tk.Tk()
root.geometry("800x500")
root.title("تصنيف الأخبارالعربيه")

# Setting items in the user interface
label1 = tk.Label(root, text="محلل تصنيف الأخبار", font=("Helvetica", 14))
label1.pack(pady=10)

label2 = tk.Label(root, text="أدخل النص:", font=("Helvetica", 12))
label2.pack(pady=5)

entry = tk.Entry(root, font=("Helvetica", 12), width=50)
entry.pack(pady=5)

classify_button = tk.Button(root, text="تصنيف", command=classify_text, font=("Helvetica", 12), bg="#002179", fg="white")
classify_button.pack(pady=10)

clear_button = tk.Button(root, text="مسح النص", command=clear_text, font=("Helvetica", 12), bg="#d9534f", fg="white")
clear_button.pack(pady=5)

result_label = tk.Label(root, text="", font=("Helvetica", 16), fg="blue")
result_label.pack(pady=20)

# Run the GUI application
root.mainloop()
